In [7]:
import pandas as pd
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt

INPUTS

In [2]:
# a parede é um cubo/quadrado determinado pelo tamanho do lado do cubo/quadrado e numero de dimensoes
tamanho_parede: float      = 10
numero_dimensoes: int      = 3
quantidade_particulas: int = 10

raio:  float = 1
massa: float = 1

velocidade_maxima: float = 10000

PARTICULAS

- Classe das particulas

In [39]:
class Particula:
    def __init__(self, raio: float, massa: float, vetor_posicao: list[float], vetor_velocidade: list[float]) -> None:
        self.massa = massa
        self.raio  = raio
        self.vetor_posicao    = np.array(vetor_posicao, dtype= np.float_)
        self.vetor_velocidade = np.array(vetor_velocidade, dtype= np.float_)

    @property
    def momento(self) -> npt.NDArray[np.float_]:
        return self.massa * self.vetor_velocidade
    
    @property
    def energia_cinetica(self) -> npt.NDArray[np.float_]:
        return self.massa * self.vetor_velocidade * self.vetor_velocidade
    
    def atualizar_posicao(self, tempo_decorrido: float) -> None:
        self.vetor_posicao += self.vetor_velocidade * tempo_decorrido
    
    def reflexao(self, dimensao_reflexão: int) -> None:
        """
        Edita o vetor velocidade, invertendo o sentido da velocidade na dimensão dada, como uma colisão elástica com a parede.

        Parameters
        ----------
        dimensao_reflexao : int
            0 - Inverte o sentido da velocidade em x\n
            1 - Inverte o sentido da velocidade em y\n
            2 - Inverte o sentido da velocidade em z
        """
        self.vetor_velocidade[dimensao_reflexão] = -self.vetor_velocidade[dimensao_reflexão]

    def __str__(self) -> str:
        return "Raio {}, massa {}, posicao {} e velocidade {}".format(self.raio, self.massa, self.vetor_posicao, self.vetor_velocidade)
    
    def __repr__(self) -> str:
        return "Particula({}, {}, {}, {})".format(self.raio, self.massa, self.vetor_posicao.tolist(), self.vetor_velocidade.tolist())

- RNG
- Criador de vetor aleatorio (com dimensoes certas e sem zeros)

In [41]:
# o RNG é um gerador aleatorio de float entre [0, 1)
rng = np.random.default_rng()

def vetor_aleatorio(valor_minimo: float, valor_maximo: float, dimensoes: int) -> npt.NDArray[np.float_]:
    """Gera um vetor (lista) com as dimensões dadas sem zeros, para não ter problemas como iniciar já colidindo com a parede"""
    vetor_zero_a_um = rng.random(dimensoes)
    
    while min(vetor_zero_a_um) == 0:
        vetor_zero_a_um[vetor_zero_a_um == 0] = rng.random() # remapeia valores nulos
    
    vetor_aleatorio = (valor_maximo - valor_minimo) * vetor_zero_a_um + valor_minimo

    return vetor_aleatorio

CRIADOR DE LISTA DE PARTICULAS

- Calculador de momento total de uma lista de particulas

In [5]:
def momento_total_lista(lista_de_particulas: list[Particula]) -> npt.NDArray[np.float_]:
    momento_total = np.zeros(numero_dimensoes)
    
    for particula in lista_de_particulas:
        momento_total += particula.momento
        
    return momento_total

- Criador de partículas
- Criador da lista de partículas

In [42]:
# criador de particula
def criador_de_particula(raio: float, massa: float) -> Particula:
    posicao_criada = vetor_aleatorio(0, tamanho_parede, numero_dimensoes)
    velocidade_criada = vetor_aleatorio(-velocidade_maxima, velocidade_maxima, numero_dimensoes) # velocidades podem ser positivas ou negativas

    particula_criada = Particula(raio, massa, posicao_criada, velocidade_criada)
    return particula_criada


# criador da lista de partículas
def criar_particulas_iniciais(quantidade_particulas: int, raio: float, massa: float) -> list[Particula]: 
    # primeiro monta uma lista inicial das particulas
    lista_particulas = [criador_de_particula(raio, massa) for i in range(quantidade_particulas)]

    # depois calculamos o momento total das particulas (exceto a ultima)
    momento_total = momento_total_lista(lista_particulas[:-1])
    
    # e fazemos as velocidades da ultima particula equilibrar para o momento total ser zero 
    ultima_particula = lista_particulas[-1]
    velocidade_ultima_particula = - momento_total / ultima_particula.massa
    ultima_particula.vetor_velocidade = velocidade_ultima_particula

    return lista_particulas
        

FUNÇÃO DAS COLISÕES

As colisões podem ser calculadas levando o problema para 1 dimensão (linha da colisão) e retornando para as N dimensões.
em 1D, as velocidades depois de uma colisão são dadas por:

v1f = [ (m1 - m2) v1 -   2 m2    v2 ] / (m1 + m2)

v1f = [   2 m1    v1 - (m1 - m2) v2 ] / (m1 + m2)

No entanto, é preciso notar que todas essas velocidades são no sentido da normal da colisão e afetam somente a velocidade nesse sentido.

v_normal = vetor_velocidade . vetor_normal

vetor_velocidade_novo = vetor_velocidade + (v_normal - v_normal_novo) * vetor_normal

In [54]:
def vetor_normal_colisao(particula_1: Particula, particula_2: Particula) -> npt.NDArray[np.float_]:
    """
    Não usar fora da colisão!\n
    Na colisão, a magnetude do vetor distância é a soma dos raios, simplificando o cálculo do vetor normal.\n
    
    Returns
    -------
    vetor_normal: ndarray[float_]
        Nota: o vetor gerado aponta para a fora da superfície da particula_1!\n
        normal = vetor_distancia / (soma_dos_raios)
    """
    posicao_1 = particula_1.vetor_posicao
    posicao_2 = particula_2.vetor_posicao

    raio_1 = particula_1.raio
    raio_2 = particula_2.raio
    
    vetor_distancia = posicao_2 - posicao_1
    modulo_vetor_distancia = raio_1 + raio_2
    
    vetor_normal = vetor_distancia / modulo_vetor_distancia

    return vetor_normal

In [55]:
def velocidades_apos_colisao(massas: list[float], velocidades_normais: list[np.float_]) -> list[np.float_]:
    """
    No momento da colisão, podemos decompor as velocidades das partículas em suas componentes normal ao impacto (v1 e v2) e tangencial. Somente a velocidade normal é alterada.\n
    v1_nova = [ (m1 - m2) v1 -   2 m2    v2 ] / (m1 + m2)\n
    v2_nova = [   2 m1    v1 - (m1 - m2) v2 ] / (m1 + m2)

    Returns
    -------
    lista_velocidades_novas: list[float_]
        É uma lista formada pelas velocidades normais novas calculadas, na forma  [v1_nova, v2_nova]
    """
    m1, m2 = massas
    v1, v2 = velocidades_normais
    
    diff_massa = m1 - m2
    soma_massa = m1 + m2

    velocidade_1_nova = (diff_massa * v1 -    2*m2    * v2) / soma_massa
    velocidade_2_nova = (   2*m1    * v1 - diff_massa * v2) / soma_massa

    return [velocidade_1_nova, velocidade_2_nova]

In [56]:
def colisao(particula_1: Particula, particula_2: Particula) -> None:
    """
    Atualiza as velocidades das partículas durante uma colisão. Usar somente se a colisão já for confirmada.\n
    Para mudar a velocidade normal sem alterar a velocidade tangencial, o vetor da velocidade tangencial é calculado e somado com a velocidade normal nova:\n
    velocidade_tangencial = velocidade_antes - velocidade_normal\n
    velocidade_depois = velocidade_tangencial + velocidade_normal_nova\n\n
    
    velocidade_depois = velocidade_antes - velocidade_normal + velocidade_normal_nova
    """
    vetor_normal = vetor_normal_colisao(particula_1, particula_2)

    massa_1 = particula_1.massa
    massa_2 = particula_2.massa

    v1_normal = np.vdot(particula_1.vetor_velocidade, vetor_normal)
    v2_normal = np.vdot(particula_2.vetor_velocidade, vetor_normal)

    v1_normal_novo, v2_normal_novo = velocidades_apos_colisao(massas = [massa_1, massa_2], velocidades_normais = [v1_normal, v2_normal])
    
    particula_1.vetor_velocidade += - v1_normal + v1_normal_novo
    particula_2.vetor_velocidade += - v2_normal + v2_normal_novo

TESTAGEM

In [7]:
teste = Particula(1,2,np.array([1,2,3]),np.array([5,10,15]))

In [8]:
teste.reflexao(0)

In [9]:
print(teste.vetor_posicao)
teste.atualizar_posicao(1)
print(teste.vetor_posicao)

[1 2 3]
[-4 12 18]


In [10]:
print(teste.vetor_velocidade)
teste.reflexao(2)
print(teste.vetor_velocidade)

[-5 10 15]
[ -5  10 -15]
